In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from collections import  Counter
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

In [0]:
PATH = "/content/drive/My Drive/Deep_Learning/Transformer_Chatbot/cornell movie-dialogs corpus/"

In [0]:
corpus_movie_conversations = PATH+str('movie_conversations.txt')
corpus_movie_lines = PATH+str('movie_lines.txt')
max_len_sequence = 25

In [0]:
with open(corpus_movie_conversations, 'r', encoding='unicode_escape') as file:
  conversations = file.readlines()
with open(corpus_movie_lines, 'r', encoding='unicode_escape') as file:
  lines = file.readlines()

In [0]:
print(conversations[0])
print(lines[0])

In [0]:
lines_dict = {}
for line in lines:
  objects = line.split(" +++$+++ ")
  lines_dict[objects[0]] = objects[-1]

In [0]:
def remove_punc(string):
  """Return String after removing punctuation and lower case all the word """
  punctuations = '''!()-{}[];:'"\/,?<>.@#$%^&*_'''
  no_punc = ""
  for char in string:
    if char not in punctuations:
      no_punc = no_punc + char
  return no_punc.lower()

In [0]:
#Test Function
s = remove_punc('''Ravi !() is -{good}[] in Deep;:'Learning\"/,?< I'm gonna do it >@#$Yeah!%^&*_''')
print(s)

In [0]:
def create_pairs(conversations, lines_dict):
  pairs = []
  for conversation in conversations:
    conv_ids = eval(conversation.split(" +++$+++ ")[-1])
    for i in range (len(conv_ids)):
      qa_pair = []
      if i == len(conv_ids) - 1:
        break
      question = remove_punc(lines_dict[conv_ids[i]].strip())
      answer = remove_punc(lines_dict[conv_ids[i+1]].strip()) 
      qa_pair.append(question.split()[:max_len_sequence])
      qa_pair.append(answer.split()[:max_len_sequence])
      pairs.append(qa_pair)
  return pairs

In [0]:
#Test Function
pairs = create_pairs(conversations, lines_dict)
len(pairs)

In [0]:
def create_word_index(pairs, min_word_freq=5):
  '''Save the word_index_map in WORDMAP_CORPUS.json file'''
  word_freq = Counter()
  for pair in pairs:
    word_freq.update(pair[0])
    word_freq.update(pair[1])
  words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
  word_map = {k: v+1 for v, k in enumerate(words)}
  word_map['<unk>'] = len(word_map) + 1
  word_map['<start>'] = len(word_map) + 1
  word_map['<end>'] = len(word_map) + 1
  word_map['<pad>'] = 0
  print("Total words are {}".format(len(word_map)))

  with open(PATH+str("WORDMAP_CORPUS.json"), 'w') as f:
    json.dump(word_map, f)
  return (words, word_map, word_map['<unk>'], word_map['<start>'], word_map['<end>'], word_map['<pad>'])

In [0]:
#Test function
words, word_map, word_map['<unk>'], word_map['<start>'], word_map['<end>'], word_map['<pad>'] = create_word_index(pairs)

In [0]:
def encode_questions(words, word_map):
  enc_c = [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<pad>']] * (max_len_sequence - len(words))
  return enc_c

In [0]:
def encode_reply(words, word_map):
  enc_c = [word_map['<start>']] + [word_map.get(word, word_map['<unk>']) for word in words] + [word_map['<end>']] + [word_map['<pad>']] * (max_len_sequence - len(words))
  return enc_c

In [0]:
#Test Function
encode_questions(words[0], word_map)

In [0]:
#Test Function
encode_reply(words[1], word_map)

In [0]:
def to_save_pairs_encoder(pairs, word_map):
  pairs_encoded = []
  for pair in pairs:
    ques = encode_questions(pair[0], word_map)
    reply = encode_reply(pair[1], word_map)
    pairs_encoded.append([ques, reply])
  print("Length of Encoded pairs {}".format(len(pairs_encoded))) 
  with open(PATH+str("pairs_encoded.json"), 'w') as f:
    json.dump(pairs_encoded, f)
  print("File Succesfully Saved at {}".format(PATH))

In [0]:
# Test Function
to_save_pairs_encoder(pairs, word_map)

In [0]:
class Dataset(Dataset):

    def __init__(self):

        self.pairs = json.load(open(PATH+str("pairs_encoded.json")))
        self.dataset_size = len(self.pairs)

    def __getitem__(self, i):
        
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
            
        return question, reply

    def __len__(self):
        return self.dataset_size

# Data Loading and Masking

In [0]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size = 100, 
                                           shuffle=True, 
                                           pin_memory=True)

In [0]:
def to_create_masks(question, reply_input, reply_target):
    
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)
    
    question_mask = question!=0
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)         # (batch_size, 1, 1, max_words)
     
    reply_input_mask = reply_input!=0
    reply_input_mask = reply_input_mask.unsqueeze(1)  # (batch_size, 1, max_words)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data) 
    reply_input_mask = reply_input_mask.unsqueeze(1) # (batch_size, 1, max_words, max_words)
    reply_target_mask = reply_target!=0              # (batch_size, max_words)
    
    return question_mask, reply_input_mask, reply_target_mask

# Embedding and Positional Encoding

In [0]:
class Embeddings(nn.Module):
    """
    Implements embeddings of the words and adds their positional encodings. 
    """
    def __init__(self, vocab_size, d_model, max_len = 50):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = self.create_positinal_encoding(max_len, self.d_model)
        self.dropout = nn.Dropout(0.1)
        
    def create_positinal_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model).to(device)
        for pos in range(max_len):   # for each position of the word
            for i in range(0, d_model, 2):   # for each dimension of the each position
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
        pe = pe.unsqueeze(0)   # include the batch size
        return pe
        
    def forward(self, encoded_words):
        embedding = self.embed(encoded_words) * math.sqrt(self.d_model)
        embedding += self.pe[:, :embedding.size(1)]   # pe will automatically be expanded with the same batch size as encoded_words
        embedding = self.dropout(embedding)
        return embedding

# Multihead Attention

In [0]:
class MultiHeadAttention(nn.Module):
    
    def __init__(self, heads, d_model):
        
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
        
    def forward(self, query, key, value, mask):
        """
        query, key, value of shape: (batch_size, max_len, 512)
        mask of shape: (batch_size, 1, 1, max_words)
        """
        # (batch_size, max_len, 512)
        query = self.query(query)
        key = self.key(key)        
        value = self.value(value)   
        
        # (batch_size, max_len, 512) --> (batch_size, max_len, h, d_k) --> (batch_size, h, max_len, d_k)
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3) 
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)          
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)     
        
        # (batch_size, h, max_len, d_k) matmul (batch_size, h, d_k, max_len) --> (batch_size, h, max_len, max_len)
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(query.size(-1))        
        scores = scores.masked_fill(mask == 0, -1e9)       
        weights = F.softmax(scores, dim = -1)           # (batch_size, h, max_len, max_len)        
        weights = self.dropout(weights)      
        # (batch_size, h, max_len, max_len) matmul (batch_size, h, max_len, d_k) --> (batch_size, h, max_len, d_k)
        context = torch.matmul(weights, value)        
        # (batch_size, h, max_len, d_k) --> (batch_size, max_len, h, d_k) --> (batch_size, max_len, h * d_k)
        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k) 
        # (batch_size, max_len, h * d_k)
        interacted = self.concat(context)
        
        return interacted 

In [0]:
class FeedForward(nn.Module):

    def __init__(self, d_model, middle_dim = 2048):
        super(FeedForward, self).__init__()
        
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

# Encoder Layer

In [0]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

# Decoder Layer

In [0]:
class DecoderLayer(nn.Module):
    
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded

# Transformer

In [0]:
class Transformer(nn.Module):
    
    def __init__(self, d_model, heads, num_layers, word_map):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = len(word_map)
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)
        
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings
    
    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
        
    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out

# Optimizer

In [0]:
class AdamWarmup:
    
    def __init__(self, model_size, warmup_steps, optimizer):
        
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
        
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
        
    def step(self):
        # Increment the number of steps each time we call the step function
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        # update the learning rate
        self.lr = lr
        self.optimizer.step()       

# Loss

In [0]:
class LossWithLS(nn.Module):

    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size
        
    def forward(self, prediction, target, mask):
        """
        prediction of shape: (batch_size, max_words, vocab_size)
        target and mask of shape: (batch_size, max_words)
        """
        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss

# Hyper parameters

In [0]:
#Hyper parameters
d_model = 512
heads = 8
num_layers = 6
epochs = 25
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open(PATH+str("WORDMAP_CORPUS.json"), 'r') as j:
  word_map = json.load(j)

transformer = Transformer(d_model = d_model, heads = heads, num_layers = num_layers, word_map = word_map)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(len(word_map), 0.1)

/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


# Training

In [0]:
def train(transformer, train_loader, criterion, epoch):

  transformer.train()
  sum_loss = 0
  count = 0
  for i, (question, reply) in enumerate(train_loader):
    samples = question.shape[0]
    question = question.to(device)
    reply = reply.to(device)

    reply_input = reply[:, :-1]
    reply_target = reply[:, 1:]

    question_mask, reply_input_mask, reply_target_mask = to_create_masks(question, reply_input, reply_target)
    
    #Run through transformer
    out = transformer(question, question_mask, reply_input, reply_input_mask)
    loss = criterion(out, reply_target, reply_target_mask)

    #Backpropagation
    transformer_optimizer.optimizer.zero_grad()
    loss.backward()
    transformer_optimizer.step()

    sum_loss += loss.item() * samples
    count += samples

    if (i % 100 == 0):
      print("Epoch [{}] [{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

# Testing

In [0]:
def evaluate(transformer, question, question_mask, max_len, word_map):
    """
    Performs Greedy Decoding with a batch size of 1
    """
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['<start>']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim = 1)
        next_word = next_word.item()
        if next_word == word_map['<end>']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)   # (1,step+2)
        
    # Construct Sentence
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words if w not in {word_map['<start>']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

# Training the model

In [0]:
model_path = "/content/drive/My Drive/Deep_Learning/Transformer_Chatbot/"

In [0]:
for epoch in range(epochs):
  train(transformer, train_loader, criterion, epoch)
  state = {'epoch': epoch, 'transformer': transformer, 'transformer_optimizer': transformer_optimizer}
  torch.save(state, model_path+'checkpoint_'+ str(epoch) + '.pth.tar')

# Testing the model

In [0]:
checkpoint = torch.load(model_path+'checkpoint_6.pth.tar')
transformer = checkpoint['transformer']

In [0]:
while(1):
  question = input("Question: ")
  if question == 'quit':
    break
  max_len = input("Maximum Reply Length: ")
  enc_qus = [word_map.get(word, word_map['<unk>']) for word in question.split()]
  question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
  question_mask = (question!=0).to(device).unsqueeze(1).unsqueeze(1)  
  sentence = evaluate(transformer, question, question_mask, int(max_len), word_map)
  print(sentence)